# OCSC Whisper Evaluation

## Overview
This notebook evaluates baseline Whisper-Medium against our fine-tuned checkpoint on the OCSC test set. We compute WER/CER metrics overall and stratified by age group and task type, then perform error analysis to understand where fine-tuning helps or hurts.

**Models compared:**
- Baseline: `openai/whisper-medium.en` (zero-shot)
- Fine-tuned: Checkpoint from W&B artifact (0.25 epochs on OCSC)

**Metrics:** Word Error Rate (WER), Character Error Rate (CER)

In [ ]:
!pip install "numpy<2.0.0" \
            "transformers==4.45.2" \
            "datasets==2.20.0" \
            "evaluate==0.4.2" \
            "huggingface_hub==0.26.2" \
            "soundfile==0.12.1" \
            "wandb<0.18"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 165.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 132.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 131.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 119.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully unin

In [ ]:
import numpy as np

print("NumPy version:", np.__version__)

NumPy version: 1.26.4


### GPU Verification
Confirm CUDA availability for inference acceleration.

### Additional Dependencies
Install evaluation metrics and remove peft to avoid import conflicts.

### Environment Setup
Import libraries, mount Google Drive for data access, and configure device.

In [ ]:
!nvidia-smi

# Pin versions to avoid the trainer/peft/transformers mismatch hell
# !pip -q install transformers==4.45.2 datasets==2.20.0 evaluate==0.4.2 \
#                huggingface_hub==0.26.2 soundfile==0.12.1 wandb==0.17.0

!pip -q install evaluate==0.4.2 soundfile==0.12.1
# We don't need PEFT here, but uninstall it so nothing tries to import it.
!pip -q uninstall -y peft

import os, re, io, subprocess, shutil, random
from pathlib import Path

import numpy as np
import pandas as pd
import soundfile as sf
import torch
from tqdm import tqdm
import evaluate

from google.colab import drive
drive.mount("/content/drive")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Wed Dec 10 22:34:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

### Configure Data Paths
Set up paths to:
- Preprocessed manifests from Google Drive
- OCSC audio files (downloaded from HuggingFace)

### Load Preprocessed Manifests
Copy the latest manifest CSVs from Drive and load train/dev/test splits into DataFrames.

In [ ]:
# Base paths
BASE_DRIVE = Path("/content/drive/MyDrive")
CONV_ROOT  = BASE_DRIVE / "ocsc_converted"   # from Notebook A
DATA       = Path("/content/data")
RAW        = DATA / "ocsc_raw"
AUDIO_ROOT = RAW / "Eng-NA" / "OCSC"

DATA.mkdir(parents=True, exist_ok=True)
RAW.mkdir(parents=True, exist_ok=True)

print("CONV_ROOT:", CONV_ROOT)
print("AUDIO_ROOT:", AUDIO_ROOT)

# Pick latest converted manifest folder
stamps = [p for p in CONV_ROOT.iterdir() if p.is_dir()]
assert stamps, f"No converted folders found under {CONV_ROOT}"
STAMP_DIR = sorted(stamps, key=lambda p: p.name)[-1]
print("Using manifests from:", STAMP_DIR)

MANI = DATA / "manifests"
MANI.mkdir(parents=True, exist_ok=True)
for name in ["ocsc_train.csv", "ocsc_dev.csv", "ocsc_test.csv", "ocsc_manifest_utterances.csv"]:
    src = STAMP_DIR / name
    if src.exists():
        shutil.copy2(src, MANI / name)

df_train = pd.read_csv(MANI / "ocsc_train.csv")
df_dev   = pd.read_csv(MANI / "ocsc_dev.csv")
df_test  = pd.read_csv(MANI / "ocsc_test.csv")

print("Train rows:", len(df_train))
print("Dev rows:",   len(df_dev))
print("Test rows:",  len(df_test))
print("Dev columns:", df_dev.columns.tolist())
df_dev.head()

CONV_ROOT: /content/drive/MyDrive/ocsc_converted
AUDIO_ROOT: /content/data/ocsc_raw/Eng-NA/OCSC
Using manifests from: /content/drive/MyDrive/ocsc_converted/20251201-213109
Train rows: 96908
Dev rows: 11230
Test rows: 25408
Dev columns: ['session_id', 'age_folder', 'audio_path', 'cha_path', 'speaker_id', 'age_years', 'age_bucket', 'task', 'start_s', 'end_s', 'text', 'norm_text', 'dur_s']


,session_id,age_folder,audio_path,cha_path,speaker_id,age_years,age_bucket,task,start_s,end_s,text,norm_text,dur_s
0,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,9.835,10.478,hi . 9835_10478,hi,0.643
1,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,31.617,32.736,Margaret . 31617_32736,margaret,1.119
2,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,46.921,48.127,yellow . 46921_48127,yellow,1.206
3,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,Alphabet,66.370,67.501,apple . 66370_67501,apple,1.131
4,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,Alphabet,68.048,71.279,a kid . 68048_71279,a kid,3.231


In [ ]:
from huggingface_hub import snapshot_download

if not AUDIO_ROOT.exists() or not any(AUDIO_ROOT.rglob("*.wav")):
    print("Downloading OCSC audio tree from NolanChai/childes-ocsc...")
    snapshot_download(
        repo_id="NolanChai/childes-ocsc",
        repo_type="dataset",
        local_dir=str(RAW),
        local_dir_use_symlinks=False,
    )

print("Audio root exists:", AUDIO_ROOT.exists())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 305 files:   0%|          | 0/305 [00:00<?, ?it/s]

4006.wav:   0%|          | 0.00/175M [00:00<?, ?B/s]

4003.wav:   0%|          | 0.00/186M [00:00<?, ?B/s]

4005.wav:   0%|          | 0.00/170M [00:00<?, ?B/s]

...

9040.mp3:   0%|          | 0.00/28.3M [00:00<?, ?B/s]

9042.mp3:   0%|          | 0.00/22.7M [00:00<?, ?B/s]

9043.mp3:   0%|          | 0.00/23.4M [00:00<?, ?B/s]

9044.mp3:   0%|          | 0.00/28.1M [00:00<?, ?B/s]

Audio root exists: True


### Audio Loading Utilities

**`resolve_audio_path`**: Robustly resolve manifest paths to actual audio files, handling path mismatches between preprocessing and evaluation environments.

**`load_clip_ffmpeg`**: Extract audio segments using ffmpeg:
- Slice by start/end timestamps
- Resample to 16kHz mono
- Return as PyTorch tensor

In [ ]:
AUDIO_EXTS = [".wav", ".mp3"]

def resolve_audio_path(p_str: str) -> str:
    """
    fix audio paths
    """
    p = Path(p_str)
    if p.exists():
        return str(p)

    m = re.search(r"(Eng-NA/OCSC/.+)$", str(p))
    if m:
        rel = Path(m.group(1)).relative_to("Eng-NA/OCSC")
        cand = AUDIO_ROOT / rel
        if cand.exists():
            return str(cand)
        for ext in AUDIO_EXTS:
            alt = cand.with_suffix(ext)
            if alt.exists():
                return str(alt)

    for ext in AUDIO_EXTS:
        alt = p.with_suffix(ext)
        if alt.exists():
            return str(alt)

    stem = p.stem
    hits = list(AUDIO_ROOT.rglob(f"{stem}.*"))
    for h in hits:
        if h.suffix.lower() in AUDIO_EXTS:
            return str(h)

    raise FileNotFoundError(f"Audio not found for: {p_str}")

def load_clip_ffmpeg(path: str, start_s: float, end_s: float, target_sr: int = 16000):
    """
    Slice [start_s, end_s) with ffmpeg into memory and return (waveform_tensor, sr).
    Mono float32 @ target_sr.
    """
    path = resolve_audio_path(path)
    dur = max(0.01, float(end_s) - float(start_s))

    cmd = [
        "ffmpeg", "-hide_banner", "-loglevel", "error",
        "-ss", f"{float(start_s):.3f}",
        "-i", path,
        "-t", f"{dur:.3f}",
        "-ac", "1", "-ar", str(target_sr),
        "-f", "wav", "pipe:1",
    ]
    out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    data, sr = sf.read(io.BytesIO(out.stdout), dtype="float32", always_2d=False)
    if data.ndim > 1:
        data = data.mean(axis=1)
    return torch.from_numpy(data), sr

## Load Models

### Baseline Model (Zero-Shot)
Load pretrained Whisper-Medium English without any fine-tuning.

### Fine-Tuned Model
Load our fine-tuned checkpoint from W&B artifacts. This model was trained for ~0.25 epochs on OCSC.

In [ ]:
import wandb

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

BASE_MODEL_ID = "openai/whisper-medium.en"

# ----- Baseline Whisper -----
baseline_model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_ID)
baseline_proc  = WhisperProcessor.from_pretrained(
    BASE_MODEL_ID,
    language="en",
    task="transcribe",
)
baseline_model.to(device)
baseline_model.eval()

print("Baseline model loaded:", BASE_MODEL_ID)

# ----- Fine-tuned from W&B artifact -----
import wandb
wandb.login()

api = wandb.Api()

# TODO: set this to the artifact that corresponds to your best checkpoint
FT_ARTIFACT_ID = "noulan/ocsc-whisper/model-whisper-medium-ocsc-ft-20251203-154530-phase:v2"

ft_ckpt_root = Path("/content/wandb_checkpoints")
ft_ckpt_root.mkdir(exist_ok=True)

ft_artifact = api.artifact(FT_ARTIFACT_ID, type="model")
ft_dir = Path(ft_artifact.download(root=str(ft_ckpt_root)))

print("Fine-tuned checkpoint dir:", ft_dir)
!ls -R "$ft_dir"

ft_model = WhisperForConditionalGeneration.from_pretrained(str(ft_dir))
ft_proc  = WhisperProcessor.from_pretrained(
    BASE_MODEL_ID,
    language="en",
    task="transcribe",
)
ft_model.to(device)
ft_model.eval()

# Remove any forced prompts / suppressions if you disabled them in training
ft_model.config.forced_decoder_ids = None
ft_model.config.suppress_tokens = []

print("Fine-tuned model loaded from artifact:", FT_ARTIFACT_ID)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

Baseline model loaded: openai/whisper-medium.en


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Downloading large artifact model-whisper-medium-ocsc-ft-20251203-154530-phase:v2, 8732.59MB. 14 files... 
wandb:   14 of 14 files downloaded.  
Done. 0:5:47.2


Fine-tuned checkpoint dir: /content/wandb_checkpoints
/content/wandb_checkpoints:
added_tokens.json	normalizer.json		 tokenizer_config.json
config.json		optimizer.pt		 trainer_state.json
generation_config.json	rng_state.pth		 training_args.bin
merges.txt		scheduler.pt		 vocab.json
model.safetensors	special_tokens_map.json
Fine-tuned model loaded from artifact: noulan/ocsc-whisper/model-whisper-medium-ocsc-ft-20251203-154530-phase:v2


### Initialize Evaluation Metrics
Load WER and CER metrics from HuggingFace evaluate library.

### Configure Column Names
Map manifest column names for audio paths, timestamps, transcriptions, and metadata.

### Evaluation Function

**`run_eval_on_df`**: Run inference on a DataFrame subset and compute WER/CER.
- Batched inference for efficiency
- Returns predictions, references, and aggregate metrics

In [ ]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 111.4 MB/s eta 0:00:00


In [ ]:
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# Column names based on your manifests from Notebook A
AUDIO_COL = "audio_path"
START_COL = "start_s"
END_COL   = "end_s"
TEXT_COL  = "norm_text"

AGE_COL   = "age_years" if "age_years" in df_dev.columns else None
TASK_COL  = "task" if "task" in df_dev.columns else (
    "task_tag" if "task_tag" in df_dev.columns else None
)
print("Using columns:",
      "AUDIO_COL =", AUDIO_COL,
      "START_COL =", START_COL,
      "END_COL   =", END_COL,
      "TEXT_COL  =", TEXT_COL,
      "AGE_COL   =", AGE_COL,
      "TASK_COL  =", TASK_COL)

def run_eval_on_df(model, processor, df, max_items=None, batch_size=4, desc="Eval"):
    """
    Evaluate WER/CER for a given model+processor on a subset of df.
    Assumes df has: AUDIO_COL, START_COL, END_COL, TEXT_COL.
    Returns: dict with wer, cer, preds, refs.
    """
    model.eval()
    indices = list(range(len(df)))
    if max_items is not None:
        indices = indices[:max_items]

    preds, refs = [], []

    for i in tqdm(range(0, len(indices), batch_size), desc=desc):
        idxs = indices[i:i + batch_size]
        waves = []
        texts = []

        for j in idxs:
            row = df.iloc[j]
            y, sr = load_clip_ffmpeg(
                row[AUDIO_COL],
                float(row[START_COL]),
                float(row[END_COL]),
            )
            waves.append(y.numpy())
            texts.append(row[TEXT_COL])

        # Prepare input features
        inputs = processor.feature_extractor(
            waves,
            sampling_rate=sr,
            return_tensors="pt",
        ).to(device)

        with torch.no_grad():
            gen_ids = model.generate(
                inputs.input_features,
                max_length=225,
            )

        pred_str = processor.tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        preds.extend(pred_str)
        refs.extend(texts)

    wer = wer_metric.compute(predictions=preds, references=refs)
    cer = cer_metric.compute(predictions=preds, references=refs)
    return {"wer": wer, "cer": cer, "preds": preds, "refs": refs}


Using columns: AUDIO_COL = audio_path START_COL = start_s END_COL   = end_s TEXT_COL  = norm_text AGE_COL   = age_years TASK_COL  = task


## Run Main Evaluation

Compare baseline vs fine-tuned model on the dev set. Report overall WER and CER.

In [ ]:
# Choose evaluation split
df_eval = df_dev   # or df_test

# Limit utterances if needed (None = use full set)
MAX_ITEMS = None   # e.g. 2000 for a quick-ish run

print("=== Baseline (openai/whisper-medium.en) on dev ===")
base_res = run_eval_on_df(
    baseline_model,
    baseline_proc,
    df_eval,
    max_items=MAX_ITEMS,
    batch_size=4,
    desc="Baseline eval",
)
print(f"Baseline WER: {base_res['wer']:.4f} | CER: {base_res['cer']:.4f}")


=== Baseline (openai/whisper-medium.en) on dev ===


Baseline eval: 100%|██████████| 2808/2808 [1:05:24<00:00,  1.40s/it]


Baseline WER: 0.6576 | CER: 0.4146

=== Fine-tuned (OCSC) on dev ===


Fine-tuned eval:   2%|▏         | 61/2808 [05:53<4:25:26,  5.80s/it]


KeyboardInterrupt: 

In [ ]:
print("\n=== Fine-tuned (OCSC) on dev ===")
ft_res = run_eval_on_df(
    ft_model,
    ft_proc,
    df_eval,
    max_items=4000,
    batch_size=4,
    desc="Fine-tuned eval",
)
print(f"Fine-tuned WER: {ft_res['wer']:.4f} | CER: {ft_res['cer']:.4f}")


=== Fine-tuned (OCSC) on dev ===


Fine-tuned eval: 100%|██████████| 1000/1000 [1:42:06<00:00,  6.13s/it]


Fine-tuned WER: 30.0023 | CER: 29.8831


## Stratified Analysis

### WER by Age Group
Break down performance by child age to see if fine-tuning helps certain age groups more.

In [ ]:
def bucket_age(age):
  if pd.isna(age):
    return None
  if age < 6:
    return "4–5"
  elif age < 8:
    return "6–7"
  else:
    return "8–9"

if AGE_COL is not None:
  df_eval_age = df_eval.copy()
  df_eval_age["age_bucket"] = df_eval_age[AGE_COL].apply(bucket_age)

  buckets = [b for b in df_eval_age["age_bucket"].unique() if b is not None]
  print("\n=== WER by age bucket (fine-tuned model) ===")
  per_age = []

  for b in sorted(buckets):
    sub = df_eval_age[df_eval_age["age_bucket"] == b]
    if len(sub) < 50:
      continue
    res_b = run_eval_on_df(
      ft_model,
      ft_proc,
      sub,
      max_items=1000,
      batch_size=4,
      desc=f"Age {b}",
    )
    print(f"Age {b}: WER={res_b['wer']:.4f}, CER={res_b['cer']:.4f}, n={len(sub)}")
    per_age.append((b, res_b["wer"], res_b["cer"], len(sub)))
else:
  print("No age column found; skipping age-bucket analysis.")



=== WER by age bucket (fine-tuned model) ===


Age 4–5: 100%|██████████| 250/250 [24:44<00:00,  5.94s/it]


Age 4–5: WER=36.4588, CER=36.4724, n=1581


Age 6–7: 100%|██████████| 250/250 [25:15<00:00,  6.06s/it]


Age 6–7: WER=27.1088, CER=26.7280, n=6142


Age 8–9: 100%|██████████| 250/250 [24:44<00:00,  5.94s/it]

Age 8–9: WER=28.7736, CER=28.8638, n=3507


### WER by Task Type
Analyze performance across different elicitation tasks (e.g., Alphabet, Numbers, Reading, etc.).

In [ ]:
if TASK_COL is not None:
  df_eval_task = df_eval.copy()
  tasks = sorted(df_eval_task[TASK_COL].dropna().unique())
  print(f"\n=== WER by task ({TASK_COL}) – fine-tuned model ===")
  per_task = []

  for tname in tasks:
    sub = df_eval_task[df_eval_task[TASK_COL] == tname]
    if len(sub) < 50:  # avoid tiny samples
      continue
    res_t = run_eval_on_df(
      ft_model,
      ft_proc,
      sub,
      max_items=1000,
      batch_size=4,
      desc=f"Task {tname}",
    )
    print(f"Task {tname:20s} WER={res_t['wer']:.4f}, CER={res_t['cer']:.4f}, n={len(sub)}")
    per_task.append((tname, res_t["wer"], res_t["cer"], len(sub)))
else:
  print("No task column found; skipping task-level WER.")



=== WER by task (task) – fine-tuned model ===


Task Alphabet: 100%|██████████| 250/250 [23:53<00:00,  5.74s/it]


Task Alphabet             WER=51.1472, CER=57.3703, n=1262


Task DescriptivePictures: 100%|██████████| 159/159 [16:01<00:00,  6.05s/it]


Task DescriptivePictures  WER=20.1381, CER=19.3251, n=636


...

Task Numbers              WER=64.9941, CER=59.5793, n=1385


Task Reading: 100%|██████████| 250/250 [24:03<00:00,  5.77s/it]


Task Reading              WER=10.8035, CER=10.2649, n=1003


Task Wug: 100%|██████████| 250/250 [25:23<00:00,  6.10s/it]

Task Wug                  WER=57.5899, CER=63.5034, n=1137


## Save Predictions
Export detailed predictions for offline error analysis.

In [ ]:
n_ft = len(ft_res["preds"])
print(n_ft)

4000


In [ ]:
dev_preds_df = df_eval.iloc[:n_ft].copy()

In [ ]:
dev_preds_df["baseline_pred"]  = base_res["preds"][:n_ft]
dev_preds_df["ft_pred"]        = ft_res["preds"]
dev_preds_df["reference_text"] = ft_res["refs"]

In [ ]:
out_csv = OUT_ANALYSIS_DIR / "ocsc_dev_baseline_vs_ft_subset.csv"
dev_preds_df.to_csv(out_csv, index=False)
print("Saved detailed predictions to:", out_csv)

Saved detailed predictions to: /content/drive/MyDrive/ocsc_eval_outputs/ocsc_dev_baseline_vs_ft_subset.csv


---
# Error Analysis

Analyze prediction errors to understand where fine-tuning helps or hurts.

### Load Predictions and Normalize Text
Apply Whisper's text normalizer for fair WER comparison.

In [ ]:
import pandas as pd
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

In [ ]:
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
normalizer = BasicTextNormalizer()

In [ ]:
def norm(s: str) -> str:
  if not isinstance(s, str):
    return ""
  return normalizer(s).strip()

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ocsc_eval_outputs/ocsc_dev_baseline_vs_ft_subset.csv")

In [ ]:
# normalize text
df["ref_norm"] = df["reference_text"].astype(str).map(norm)
df["baseline_norm"] = df["baseline_pred"].astype(str).map(norm)
df["ft_norm"] = df["ft_pred"].astype(str).map(norm)

### Compute Per-Utterance WER
Calculate WER for each utterance to identify improvements and regressions.

In [ ]:
def per_utt_wer(ref, hyp):
  return wer_metric.compute(predictions=[hyp], references=[ref])

In [ ]:
def per_utt_cer(ref, hyp):
  return cer_metric.compute(predictions=[hyp], references=[ref])

In [ ]:
df["wer_baseline"] = [
  per_utt_wer(r, h) for r, h in zip(df["ref_norm"], df["baseline_norm"])
]

In [ ]:
df["wer_ft"] = [
  per_utt_wer(r, h) for r, h in zip(df["ref_norm"], df["ft_norm"])
]

In [ ]:
df["delta_wer"] = df["wer_ft"] - df["wer_baseline"]

### Categorize Changes
Split utterances into improved, worsened, and unchanged categories.

In [ ]:
improved   = df[df["delta_wer"] < -0.05]
worsened   = df[df["delta_wer"] >  0.05]
unchanged  = df[(df["delta_wer"] >= -0.05) & (df["delta_wer"] <= 0.05)]

In [ ]:
print("Improved n:", len(improved))
print("Worsened n:", len(worsened))
print("Unchanged n:", len(unchanged))

Improved n: 208
Worsened n: 3213
Unchanged n: 579


### Example Comparisons
Show examples where fine-tuning improved or worsened transcription quality.

In [ ]:
def show_examples(subset, n=10, title=""):
  print(f"\n=== {title} (n={len(subset)}) ===")
  for _, row in subset.sample(min(n, len(subset)), random_state=0).iterrows():
    print("\nREF:", row["ref_norm"])
    print("BASE:", row["baseline_norm"])
    print("FT:  ", row["ft_norm"])
    print(f"WER_base={row['wer_baseline']:.2f} | WER_ft={row['wer_ft']:.2f}")

In [ ]:
show_examples(improved, 5, "Examples where FT improved")
show_examples(worsened, 5, "Examples where FT got worse")


=== Examples where FT improved (n=208) ===

REF: but we don t have it anymore
BASE: we used to have a garden in our backyard but we don t have it anymore
FT:   but we don t have it anymore
WER_base=1.29 | WER_ft=0.00

REF: because it would eat me
BASE: because they would eat me
FT:   because it would eat me
WER_base=0.20 | WER_ft=0.00

REF: and a lot of asteroids
BASE: and i love asteroids
FT:   and a lot of asteroids
WER_base=0.60 | WER_ft=0.00

REF: light
BASE: night
FT:   light
WER_base=1.00 | WER_ft=0.00

REF: hotdog flavor
BASE: the hot dog flavor
FT:   yock dog flavor
WER_base=1.50 | WER_ft=1.00

=== Examples where FT got worse (n=3213) ===

REF: what is that
BASE: what is that
FT:   what is that xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx xxx

### WER by Age and Task
Aggregate error analysis by metadata dimensions.

In [ ]:
age_col  = "age_years"  # or age_group if that’s cleaner
task_col = "task"

In [ ]:
if age_col in df.columns:
  print("\n=== Mean WER by age ===")
  print(df.groupby(age_col)[["wer_baseline", "wer_ft"]].mean())


=== Mean WER by age ===
           wer_baseline     wer_ft
age_years                         
4.0            0.655339  67.655972
5.0            0.410947  49.403816
6.0            0.602384  52.652957


In [ ]:
if task_col in df.columns:
  print("\n=== Mean WER by task ===")
  print(df.groupby(task_col)[["wer_baseline", "wer_ft"]].mean())


=== Mean WER by task ===
                     wer_baseline     wer_ft
task                                        
Alphabet                 0.641432  66.911200
DescriptivePictures      0.470710  36.060445
EndTasks                 0.966667  91.444444
ExpPictures              0.536201  44.401509
HowTo                    0.341898  35.963730
IntroRobot               0.795414  69.709436
Numbers                  0.764501  89.040757
Reading                  0.595852  29.912987
Wug                      0.762762  71.199196


### WER by Utterance Length
Analyze whether shorter or longer utterances are handled better after fine-tuning.

In [ ]:
# By utterance length (in words)
df["ref_len_words"] = df["ref_norm"].str.split().apply(len)

print("\n=== Mean WER by length bin ===")
df["len_bin"] = pd.cut(
  df["ref_len_words"],
  bins=[0, 3, 7, 15, 999],
  labels=["1–3", "4–7", "8–15", "16+"],
)

print(df.groupby("len_bin")[["wer_baseline", "wer_ft"]].mean())


=== Mean WER by length bin ===
         wer_baseline     wer_ft
len_bin                         
1–3          0.706938  76.067097
4–7          0.326212  20.627742
8–15         0.284245   8.467023
16+          0.199831   4.205973


/tmp/ipython-input-4177901580.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("len_bin")[["wer_baseline", "wer_ft"]].mean())


---
# Clustering Analysis

Use K-means clustering to identify patterns in utterances where fine-tuning changed behavior. This helps discover systematic improvements or regressions.

### Prepare Features for Clustering
Extract numeric features: utterance length, WER values, age.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from jiwer import wer as jiwer_wer
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import random

In [ ]:
BASE_DRIVE = Path("/content/drive/MyDrive")
CSV_PATH = BASE_DRIVE / "ocsc_eval_outputs" / "ocsc_dev_baseline_vs_ft_subset.csv"  # change if needed

df = pd.read_csv(CSV_PATH)
print("Loaded shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()

Loaded shape: (4000, 16)
Columns: ['session_id', 'age_folder', 'audio_path', 'cha_path', 'speaker_id', 'age_years', 'age_bucket', 'task', 'start_s', 'end_s', 'text', 'norm_text', 'dur_s', 'baseline_pred', 'ft_pred', 'reference_text']


,session_id,age_folder,audio_path,cha_path,speaker_id,age_years,age_bucket,task,start_s,end_s,text,norm_text,dur_s,baseline_pred,ft_pred,reference_text
0,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,9.835,10.478,hi . 9835_10478,hi,0.643,Bye.,bye,hi
1,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,31.617,32.736,Margaret . 31617_32736,margaret,1.119,Margaret.,smog lit tt tt tt tt tt tt tt tt tt tt tt tt t...,margaret
2,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,IntroRobot,46.921,48.127,yellow . 46921_48127,yellow,1.206,yellow,yellow oo oo oo oo oo oo oo oo oo oo oo oo oo ...,yellow
3,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,Alphabet,66.370,67.501,apple . 66370_67501,apple,1.131,Apple.,apple apple xxx xxx xxx xxx xxx xxx xxx xxx xx...,apple
4,4024,4,/content/data/ocsc_raw/Eng-NA/OCSC/4/4024.wav,/content/4024.cha,CHI_4024,4.0,4-5,Alphabet,68.048,71.279,a kid . 68048_71279,a kid,3.231,Okay.,a cake,a kid


### Compute Per-Utterance WER and Filter Changed Utterances

In [ ]:
normalizer = BasicTextNormalizer()

In [ ]:
def norm_text(s):
  if not isinstance(s, str):
    return ""
  return normalizer(s).strip()

In [ ]:
# normalize reference + predictions
df["ref_norm"] = df["reference_text"].astype(str).map(norm_text)
df["baseline_norm"] = df["baseline_pred"].astype(str).map(norm_text)
df["ft_norm"] = df["ft_pred"].astype(str).map(norm_text)

In [ ]:
df["ref_len_words"] = df["ref_norm"].str.split().apply(len)
df["baseline_len_words"] = df["baseline_norm"].str.split().apply(len)
df["ft_len_words"] = df["ft_norm"].str.split().apply(len)

In [ ]:
print("Example normalized rows:")
df[["reference_text", "ref_norm", "baseline_norm", "ft_norm"]].head()

Example normalized rows:


,reference_text,ref_norm,baseline_norm,ft_norm
0,hi,hi,bye,bye
1,margaret,margaret,margaret,smog lit tt tt tt tt tt tt tt tt tt tt tt tt t...
2,yellow,yellow,yellow,yellow oo oo oo oo oo oo oo oo oo oo oo oo oo ...
3,apple,apple,apple,apple apple xxx xxx xxx xxx xxx xxx xxx xxx xx...
4,a kid,a kid,okay,a cake


In [ ]:
def per_utt_wer(ref, hyp):
  # jiwer.wer returns a ratio i.e., WER
  return jiwer_wer(ref, hyp)

In [ ]:
N_ROWS = 500
sub_df = df if N_ROWS is None else df.iloc[:N_ROWS].copy()
print("Computing per-utterance WER (baseline / ft)...")

Computing per-utterance WER (baseline / ft)...


In [ ]:
baseline_wers = []
ft_wers = []

In [ ]:
for r, b, f in zip(sub_df["ref_norm"], sub_df["baseline_norm"], sub_df["ft_norm"]):
  baseline_wers.append(per_utt_wer(r, b))
  ft_wers.append(per_utt_wer(r, f))

In [ ]:
sub_df["wer_baseline"] = baseline_wers
sub_df["wer_ft"] = ft_wers
sub_df["delta_wer"] = sub_df["wer_ft"] - sub_df["wer_baseline"]

In [ ]:
print("Overall (mean) WERs on this subset:")
print("Baseline WER:", sub_df["wer_baseline"].mean())
print("FT WER: ", sub_df["wer_ft"].mean())
print("Delta WER: ", sub_df["delta_wer"].mean())

Overall (mean) WERs on this subset:
Baseline WER: 0.654096103896104
FT WER:  67.89072467532468
Delta WER:  67.23662857142858


In [ ]:
THRESH = 0.05  # 5 percentage points
changed = sub_df[sub_df["delta_wer"].abs() > THRESH].copy()

print("Total utterances in subset:", len(sub_df))
print("Utterances with |delta_wer| >", THRESH, ":", len(changed))

if len(changed) < 10:
    print("WARNING: very few changed utterances; clustering may be uninformative.")

Total utterances in subset: 500
Utterances with |delta_wer| > 0.05 : 458


### Run K-Means Clustering

In [ ]:
# features for clustering
num_features = [
    "ref_len_words",
    "baseline_len_words",
    "ft_len_words",
    "wer_baseline",
    "wer_ft",
    "delta_wer",
]

# age task
if "age_years" in changed.columns:
    num_features.append("age_years")

X = changed[num_features].fillna(0.0).to_numpy()

# standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature matrix shape:", X_scaled.shape)

Feature matrix shape: (458, 7)


In [ ]:
from sklearn.cluster import KMeans

N_CLUSTERS = 4

if len(changed) < N_CLUSTERS:
  print("Too few examples for", N_CLUSTERS, "clusters; reducing k.")
  N_CLUSTERS = max(1, len(changed))

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

changed["cluster"] = cluster_labels
print("Cluster counts:")
print(changed["cluster"].value_counts().sort_index())

Cluster counts:
cluster
0     93
1    206
2    155
3      4
Name: count, dtype: int64


### Analyze Cluster Characteristics

In [ ]:
group_cols = ["wer_baseline", "wer_ft", "delta_wer", "ref_len_words", "baseline_len_words", "ft_len_words"]
if "age_years" in changed.columns:
  group_cols.append("age_years")

print("\n=== Cluster summary (means) ===")
summary = changed.groupby("cluster")[group_cols].mean().round(3)
print(summary)


=== Cluster summary (means) ===
         wer_baseline   wer_ft  delta_wer  ref_len_words  baseline_len_words  \
cluster                                                                        
0               0.391   24.217     23.826          5.527               5.516   
1               0.703  123.942    123.239          1.320               1.403   
2               0.793   37.111     36.318          2.297               2.316   
3               0.500   97.000     96.500          1.500               1.500   

         ft_len_words  age_years  
cluster                           
0             122.989        4.0  
1             154.777        4.0  
2              86.239        4.0  
3             140.000        5.0  


In [ ]:
print(f"\n=== Task distribution per cluster ({TASK_COL}) ===")
task_summary = changed.pivot_table(
    index="cluster",
    columns=TASK_COL,
    values="ref_norm",
    aggfunc="count",
    fill_value=0,
)
print(task_summary)


=== Task distribution per cluster (task) ===
task     Alphabet  DescriptivePictures  EndTasks  ExpPictures  HowTo  \
cluster                                                                
0               1                   16         0           24     36   
1              46                    5         1           60     20   
2              36                    8         0           33     19   
3               2                    0         0            0      0   

task     IntroRobot  Numbers  Reading  Wug  
cluster                                     
0                 0        2        1   13  
1                 2       44        1   27  
2                 0       18        0   41  
3                 2        0        0    0  


### Inspect Cluster Examples
View sample utterances from each cluster to understand patterns.

In [ ]:
random.seed(0)

In [ ]:
def show_cluster_examples(cluster_id, n=5):
  sub = changed[changed["cluster"] == cluster_id]
  if sub.empty:
    print(f"\n=== Cluster {cluster_id}: EMPTY ===")
    return

  print(f"\n=== Cluster {cluster_id}: n={len(sub)} ===")
  print(summary.loc[cluster_id])
  if TASK_COL is not None:
    print("Top tasks:")
    print(sub[TASK_COL].value_counts().head(5))

  samples = sub.sample(min(n, len(sub)), random_state=0)

  for idx, row in samples.iterrows():
    print("\n--- Example ---")
    print("REF:", row["ref_norm"])
    print("BASE:", row["baseline_norm"])
    print("FT:  ", row["ft_norm"])
    print(f"WER_base={row['wer_baseline']:.2f} | WER_ft={row['wer_ft']:.2f} | Δ={row['delta_wer']:.2f}")
    if "age_years" in row:
      print("Age:", row["age_years"])
    if TASK_COL is not None:
      print("Task:", row[TASK_COL])

for c in sorted(changed["cluster"].unique()):
  show_cluster_examples(c, n=5)


=== Cluster 0: n=93 ===
wer_baseline            0.391
wer_ft                 24.217
delta_wer              23.826
ref_len_words           5.527
baseline_len_words      5.516
ft_len_words          122.989
age_years               4.000
Name: 0, dtype: float64
Top tasks:
task
HowTo                  36
ExpPictures            24
DescriptivePictures    16
Wug                    13
Numbers                 2
Name: count, dtype: int64

--- Example ---
REF: um one
BASE: you re really good at this this is a slide
FT:   and like oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo oo o
WER_base=5.00 | WER_ft=56.50 | Δ=51.50
Age: 4.0
Task: Wug

--- Example ---
REF: he s acting like a flyer pilot
BASE: he s acting like a flyer pi